In [1]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv -O train.csv
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv -O test.csv



--2025-07-27 19:50:57--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M  41.5MB/s    in 0.7s    

2025-07-27 19:51:00 (41.5 MB/s) - ‘train.csv’ saved [29470338/29470338]

--2025-07-27 19:51:00--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1

In [2]:
import pandas as pd

# Load the training and test data
train_df = pd.read_csv("train.csv", header=None, names=["label", "title", "description"])
test_df = pd.read_csv("test.csv", header=None, names=["label", "title", "description"])

# Optional: Combine title and description
train_df["text"] = train_df["title"] + ". " + train_df["description"]
test_df["text"] = test_df["title"] + ". " + test_df["description"]

# Convert label from 1-4 to 0-3
train_df["label"] = train_df["label"] - 1
test_df["label"] = test_df["label"] - 1

# Show example
train_df.head()



,label,title,description,text
0,2,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,2,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,2,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,2,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [3]:
!pip install transformers datasets scikit-learn gradio




In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize using truncation and padding
def tokenize_data(texts, labels):
    tokens = tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    )
    tokens["labels"] = labels
    return tokens


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from torch.utils.data import Dataset

import torch

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            key: val[idx] for key, val in self.encodings.items()
        }

    def __len__(self):
        return len(self.labels)

# Encode
train_encodings = tokenize_data(train_df["text"], list(train_df["label"]))
test_encodings = tokenize_data(test_df["text"], list(test_df["label"]))

# Dataset objects
train_dataset = NewsDataset(train_encodings, train_df["label"].tolist())
test_dataset = NewsDataset(test_encodings, test_df["label"].tolist())


In [6]:
!pip install -U transformers


In [ ]:
# Install dependencies (if not done already)
!pip install -U transformers datasets scikit-learn

# 1. Load AG News Dataset
from datasets import load_dataset
dataset = load_dataset("ag_news")

# 2. Load Tokenizer and Tokenize the dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)

# 3. Format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 4. Split the train and test sets
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

# 5. Load Model
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# 6. Define TrainingArguments and Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 7. Train the model
trainer.train()
from sklearn.metrics import accuracy_score, f1_score

# Get predictions
preds = trainer.predict(test_dataset)
pred_labels = preds.predictions.argmax(axis=1)

# Metrics
acc = accuracy_score(test_dataset["label"], pred_labels)
f1 = f1_score(test_dataset["label"], pred_labels, average="weighted")

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

In [ ]:
!pip install --upgrade transformers



In [ ]:
import gradio as gr

def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    labels = ["World", "Sports", "Business", "Sci/Tech"]
    return labels[pred]

demo = gr.Interface(fn=classify_news, inputs="text", outputs="text")
demo.launch(share=True)
